In [ ]:
"""A comprehensive guide to grouping and aggregation with pandas."""

# Подробное руководство по группировке и агрегированию с помощью pandas

## Введение

Одна из базовых функций анализа данных - группировка и агрегирование. В некоторых случаях этого уровня анализа может быть достаточно, чтобы ответить на вопросы бизнеса. В других случаях - это может стать первым шагом в более сложном анализе. 

В pandas функцию [`groupby`](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) можно комбинировать с одной или несколькими функциями агрегирования, чтобы быстро и легко обобщать данные. Эта концепция обманчиво проста и большинство новых пользователей pandas поймут ее. Однако они удивятся тому, насколько полезными могут стать функции агрегирования для проведения сложного анализа данных. 

В этом Блокноте кратко изложены основные функции агрегирования pandas и показаны примеры более сложных настраиваемых агрегаций. Независимо от того, являетесь ли вы начинающим или опытным пользователем pandas, я думаю, вы узнаете что-то новое для себя.

Оригинал статьи Криса [тут](https://pbpython.com/groupby-agg.html).

## Агрегирование

В контексте даннной статьи *функция агрегирования* - это функция, которая принимает несколько отдельных значений и возвращает сводные данные. В большинстве случаев возвращаемые данные представляют собой одно значение.

Наиболее распространенные функции агрегирования - это *простое среднее* (simple average) или *суммирование* (summation) значений.

Далее представлен пример расчета суммарной и средней стоимости билетов для набора данных "Титаник", загруженного из пакета [seaborn](https://seaborn.pydata.org/examples/index.html).

> *15 апреля 1912 года самый большой пассажирский лайнер в истории во время своего первого рейса столкнулся с айсбергом. Когда Титаник затонул, погибли 1502 из 2224 пассажиров и членов экипажа. Эта сенсационная трагедия потрясла международное сообщество и привела к улучшению правил безопасности для судов. Одна из причин, по которой кораблекрушение привело к гибели людей, заключалась в том, что не хватало спасательных шлюпок для пассажиров и экипажа. Несмотря на то, что в выживании после затопления была определенная доля удачи, некоторые группы людей имели больше шансов выжить, чем другие*.

In [ ]:
from functools import partial

import numpy as np
import pandas as pd
import seaborn as sns

# import sidetable
from scipy.stats import mode, skew, trim_mean
from sparklines import sparklines

df = sns.load_dataset("titanic")

Каждая строка набора данных представляет одного человека. Столбцы описывают различные атрибуты, включая то, выжили ли они (`survived`), их возраст (`age`), класс пассажира (`pclass`), пол (`sex`) и стоимость проезда (`fare`).

In [ ]:
df.head()

In [ ]:
# сумма и среднее по столбцу стоимости билета,
# здесь передаем список агрегирующих функций
df["fare"].agg(["sum", "mean"])

Эта простая концепция - необходимый строительный блок для более сложного анализа. 

Одна из областей, которую необходимо обсудить, - это то, что существует несколько способов вызова функции агрегирования. Как показано выше, вы можете передать *список функций* для применения к одному или нескольким столбцам данных.

Что, если вы хотите выполнить анализ только подмножества столбцов? 

Есть два других варианта агрегирования: *использование словаря* и *именованное агрегирование* (named aggregation).

Использование словаря:

In [ ]:
df.agg({"fare": ["sum", "mean"], "sex": ["count"]})

Использование кортежей (именованное агрегирование):

In [ ]:
df.agg(fare_sum=("fare", "sum"), fare_mean=("fare", "mean"), sex_count=("sex", "count"))

Важно знать об этих параметрах и понимать, какой из них и когда использовать.

> *Я предпочитаю использовать словари для агрегирования.*

Подход с кортежами ограничен возможностью применять только одно агрегирование за раз к определенному столбцу. Если мне нужно переименовать столбцы, я буду использовать функцию [`rename`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) после завершения агрегации. В некоторых случаях подход со списком является более рациональным. Тем не менее, я повторю, что, на мой взгляд, словарный подход обеспечивает наиболее надежный способ для большинства ситуаций.

## Groupby

Теперь, когда мы знаем, как использовать агрегацию, мы можем объединить это с [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) для резюмирования данных.

### Основы математики

Наиболее распространенными встроенными функциями агрегирования являются базовые математические функции, включая *сумму* (sum), *среднее значение* (mean), *медианное значение* (median), *минимум* (minimum), *максимум* (maximum), *стандартное отклонение* (standard deviation), *дисперсию* (variance), *среднее абсолютное отклонение* (mean absolute deviation) и *произведение* (product).,

Мы можем применить все эти функции к `fare` (стоимости проезда) при группировке по `embark_town` (городу посадки на корабль):

In [ ]:
agg_func_math = {
    "fare": ["sum", "mean", "median", "min", "max", "std", "var", "mad", "prod"]
}

In [ ]:
# df.groupby(["embark_town"]).agg(agg_func_math).round(2)
df.groupby("embark_town")["fare"].apply(lambda x: np.mean(np.abs(x - x.mean())))

Это все относительно простая математика.

Кстати, я не нашел подходящего варианта использования функции [`prod`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.prod.html), которая вычисляет произведение всех значений в группе, и включил ее для полноты картины.

Еще один полезный трюк - использовать [`describe`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) для одновременного выполнения нескольких встроенных агрегаторов:

In [ ]:
agg_func_describe = {"fare": ["describe"]}

In [ ]:
df.groupby(["embark_town"]).agg(agg_func_describe).round(2)

### Подсчёт

После базовой математики подсчёт (counting) является следующим наиболее распространенным агрегированием, которое я выполняю для сгруппированных данных.

Он несколько сложнее, чем простая математика. Вот три примера подсчета:

In [ ]:
agg_func_count = {"embark_town": ["count", "nunique", "size"]}

In [ ]:
df.groupby(["deck"]).agg(agg_func_count)  # статистика по палубам Титаника

> Главное отличие, о котором следует помнить, заключается в том, что `count` не включает значения `NaN`, тогда как `size` их включает. В зависимости от набора данных это различие может оказаться полезным. 

Кроме того, функция `nunique` исключит значения `NaN` из уникальных счетчиков. 

Продолжайте читать, чтобы увидеть пример того, как включить `NaN` в подсчет уникальных значений.

### Первый и последний

В следующем примере мы можем выбрать самую высокую и самую низкую стоимость билета в зависимости от города, в котором совершили посадку пассажиры Титаника. 

Следует помнить один важный момент: вы должны сначала отсортировать данные, если хотите, чтобы в качестве `first` (первого) и `last` (последнего) были выбраны максимальное и минимальное значения.

In [ ]:
agg_func_selection = {"fare": ["first", "last"]}

In [ ]:
df.sort_values(by=["fare"], ascending=False).groupby(["embark_town"]).agg(
    agg_func_selection
)

В приведенном выше примере я бы рекомендовал использовать `max` и `min`, но для полноты картины включил `first` и `last`. В других приложениях (например, при анализе временных рядов) вы можете выбрать значения `first` и `last` для дальнейшего анализа.

Другой подход к выбору - использовать `idxmax` и `idxmin` для выбора значения индекса, соответствующего максимальному или минимальному значениям.

In [ ]:
agg_func_max_min = {"fare": ["idxmax", "idxmin"]}

In [ ]:
df.groupby(["embark_town"]).agg(agg_func_max_min)

Можем проверить результаты:

In [ ]:
df.loc[[258, 378]]

Вот еще один трюк, который можно использовать для просмотра строк с максимальной стоимостью проезда (`fare`):

In [ ]:
print(df.loc[df.groupby("class")["fare"].idxmax()])

Приведенный выше пример - одно из тех мест, где агрегирование на основе списка является полезным.

### Другие библиотеки

Вы не ограничены функциями агрегирования в pandas. К примеру, можно использовать функции статистики из [`scipy`](https://docs.scipy.org/doc/scipy/reference/stats.html) или [`numpy`](https://numpy.org/doc/stable/reference/routines.statistics.html).

Вот пример расчета *моды* (`mode`) и *асимметрии* (`skew`) данных для стоимости проезда.

In [ ]:
agg_func_stats = {"fare": [skew, mode, pd.Series.mode]}

In [ ]:
agg_func_text = {"embarked": ["nunique", "count", "first"]}

In [ ]:
df.groupby(["class"]).agg(agg_func_text)

In [ ]:
q_25 = partial(
    pd.Series.quantile, q=0.25
)  # возвращает обортку над pd.Series.quantile()

In [ ]:
# пойдет в наименование будущего столбца
# q_25.__name__ = "25%"

Затем мы определяем нашу собственную функцию (которая представляет собой небольшую обертку для [`quantile`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.quantile.html)):

In [ ]:
def percentile_25(a_var: pd.Series) -> float:  # type: ignore
    """Возвращает значение 25-го перцентиля для ряда данных."""
    return a_var.quantile(0.25)

Далее определяем лямбда-функцию и даем ей имя:

In [ ]:
def lambda_25(b_var: pd.Series) -> float:  # type: ignore
    """Возвращает 25-й перцентиль значений."""
    return b_var.quantile(0.25)

Затем задаем встроенную (inline) лямбду и формируем словарь:

In [ ]:
agg_func = {"fare": [q_25, percentile_25, lambda_25, lambda x: x.quantile(0.25)]}

In [ ]:
df.groupby(["embark_town"]).agg(agg_func).round(2)

Как видите, результаты одинаковые, но названия столбцов немного отличаются. Это область предпочтений программистов, но я рекомендую ознакомиться с вариантами, поскольку вы встретите большинство из них в онлайн-решениях.

> *Я предпочитаю использовать собственные функции или встроенные (inline) лямбды.*

Как и во многих других областях программирования - это элемент стиля и предпочтений, но я рекомендую вам выбрать один или два подхода и придерживаться их для единообразия.

### Примеры пользовательских функций 

Как показано выше, существует несколько подходов к разработке пользовательских функций агрегирования.

В большинстве случаев функции представляют собой легкие обертки (wrappers) для встроенных функций pandas. Они нужны, т.к. нет возможности передать аргументы в агрегаты (aggregations).

Следующие примеры должны пояснить этот момент.

Если вы хотите подсчитать количество нулевых значений, вы можете использовать эту [функцию](https://medium.com/escaletechblog/writing-custom-aggregation-functions-with-pandas-96f5268a8596):

In [ ]:
def count_nulls(c_var: pd.Series) -> int:  # type: ignore
    """Подсчитывает количество пропущенных (NaN) значений в серии данных."""
    return c_var.size - c_var.count()

Если вы хотите включить значения `NaN` в свои уникальные счетчики, вам необходимо указать параметр `dropna=False` у функции [`nunique`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.nunique.html).

In [ ]:
def unique_nan(d_var: pd.Series) -> int:  # type: ignore
    """Возвращает количество уникальных значений в серии данных."""
    return d_var.nunique(dropna=False)

Вот результат применения всех функций:

In [ ]:
agg_func_custom_count = {
    "embark_town": ["count", "nunique", "size", unique_nan, count_nulls, set]
}

df.groupby(["deck"]).agg(agg_func_custom_count)

Если вы хотите рассчитать *90-й процентиль*, используйте [`quantile`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.quantile.html):

In [ ]:
def percentile_90(e_var: pd.Series) -> float:  # type: ignore
    """Возвращает 90-й перцентиль значений."""
    return e_var.quantile(0.9)

Если вы хотите вычислить *усеченное среднее* (trimmed mean) значение, из которого исключен самый низкий 10-й процент, используйте функцию [`trim_mean`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.trim_mean.html) из `scipy`:

In [ ]:
def trim_mean_10(f_var: pd.Series) -> float:  # type: ignore
    """Вычисляет усечённое среднее, исключая по 10% крайних значений."""
    return trim_mean(f_var, 0.1)  # type: ignore

Если вы хотите получить наибольшее значение, независимо от порядка сортировки (см. ранее в этом Блокноте о `first` и `last`):

In [ ]:
def largest(g_var: pd.Series) -> pd.Series:  # type: ignore
    """Возвращает максимальное значение."""
    return g_var.nlargest(1)

Это эквивалентно `max`, но я приведу еще один пример с `nlargest` ниже, чтобы подчеркнуть разницу.

Ранее я уже [писал](https://pbpython.com/styling-pandas.html) о [`sparkline`](https://pypi.org/project/sparklines/). Обратитесь к этой статье за инструкциями по установке. 

Вот как включить их в агрегатную функцию для уникального представления данных:

In [ ]:
# pip install sparklines

In [ ]:
def sparkline_str(h_var: np.ndarray) -> str:
    """Возвращает текстовую sparkline-гистограмму для массива значений."""
    bins = np.histogram(h_var)[0]
    sl = "".join(sparklines(bins))
    return sl

Вот они все вместе:

In [ ]:
agg_func_largest = {"fare": [percentile_90, trim_mean_10, largest, sparkline_str]}

In [ ]:
df.groupby(["class", "embark_town"]).agg(agg_func_largest)

Функции `nlargest` и `nsmallest` могут быть полезны для резюмирования данных в различных сценариях. 

Следующий код показывает суммарную стоимость для 10 первых и 10 последних пассажиров:

In [ ]:
def top_10_sum(i_var: pd.Series) -> float:  # type: ignore
    """Возвращает сумму 10 наибольших значений."""
    return i_var.nlargest(10).sum()  # type: ignore

In [ ]:
def bottom_10_sum(j_var: pd.Series) -> float:  # type: ignore
    """Возвращает сумму 10 наименьших значений."""
    return j_var.nsmallest(10).sum()  # type: ignore

In [ ]:
agg_func_top_bottom_sum = {"fare": [top_10_sum, bottom_10_sum]}

In [ ]:
df.groupby("class").agg(agg_func_top_bottom_sum)

Использование этого подхода может быть полезно для применения [закона Парето](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%BA%D0%BE%D0%BD_%D0%9F%D0%B0%D1%80%D0%B5%D1%82%D0%BE) к вашим собственным данным.

### Пользовательские функции с несколькими столбцами

Если у вас есть сценарий, в котором небходимо запустить несколько агрегаций по столбцам, то вы можете использовать `groupby` в сочетании с `apply`, как описано в этом [ответе на stack overflow](https://stackoverflow.com/questions/14529838/apply-multiple-functions-to-multiple-groupby-columns/47103408#47103408).

Используя этот метод, вы получите доступ ко всем столбцам данных и сможете выбрать подходящий способ агрегирования для создания итогового `DataFrame` (включая наименование столбцов):

In [ ]:
# def summary(k_var: pd.DataFrame) -> pd.Series:
#     """Возвращает сумму, среднее и диапазон значений 'fare'."""
#     result = {
#         "fare_sum": k_var["fare"].sum(),
#         "fare_mean": k_var["fare"].mean(),
#         "fare_range": k_var["fare"].max() - k_var["fare"].min(),
#     }
#     return pd.Series(result).round(0)

In [ ]:
# df.groupby(['class']).apply(summary)

Использование `apply` с `groupby` дает максимальную гибкость. Однако есть и обратная сторона. Функция `apply` работает медленно, поэтому этот подход следует использовать с осторожностью.

## Работа с групповыми объектами

После группировки и агрегирования данных вы можете выполнять дополнительные вычисления для сгруппированных объектов.

В следующем примере определим, какой процент от общего количества проданных билетов можно отнести к каждой комбинации `embark_town` и `class`. 

Мы используем метод [`assign()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.assign.html) и лямбда-функцию для добавления столбца `pct_total`:

In [ ]:
df.groupby(["embark_town", "class"]).agg({"fare": "sum"}).assign(
    pct_total=lambda x: x / x.sum()
)

Следует отметить, что можно сделать проще с использованием кросс-таблицы [`pd.crosstab`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html), как описано в [статье](https://pbpython.com/pandas-crosstab.html):

In [ ]:
pd.crosstab(
    df["embark_town"], df["class"], values=df["fare"], aggfunc="sum", normalize=True
)

Пока мы говорим о `crosstab` (кросс-таблицах), полезно иметь в виду, что функции агрегации также можно комбинировать со сводными таблицами (pivot tables).

Вот небольшой пример:

In [ ]:
pd.pivot_table(
    data=df,
    index=["embark_town"],
    columns=["class"],
    aggfunc=agg_func_top_bottom_sum,  # type: ignore
)

Иногда необходимо выполнить множество группировок (multiple groupby), чтобы ответить на вопрос. Например, если мы хотим увидеть кумулятивную сумму стоимости билетов, мы можем сгруппировать и агрегировать по городу (town) и классу (class), затем сгруппировать полученный объект и вычислить кумулятивную сумму (cumulative sum):

In [ ]:
# pylint: disable=line-too-long

fare_group = df.groupby(["embark_town", "class"]).agg({"fare": "sum"})
fare_group

In [ ]:
fare_group = df.groupby(["embark_town", "class"]).agg({"fare": "sum"})
fare_group

In [ ]:
fare_group.groupby(level=0).cumsum()

Это может быть сложным для понимания. Вот краткое пояснение того, что мы делаем:

<img src="https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/multiple-groupby.png" >

### Пример с данными о продажах

В следующем примере резюмируем ежедневные данные о продажах и преобразуем их в совокупное ежедневное и ежеквартальное представление. 

Обратитесь к [статье о Grouper](https://pbpython.com/pandas-grouper-agg.html), если вы не знакомы с использованием метода [`pd.Grouper()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Grouper.html).

В этом примере мы хотим включить сумму ежедневных продаж, а также совокупную (cumulative) сумму за квартал:

In [ ]:
sales = pd.read_excel(
    "https://github.com/chris1610/pbpython/blob/master/data/2018_Sales_Total_v2.xlsx?raw=True"
)
sales.head()

In [ ]:
daily_sales = (
    sales.groupby([pd.Grouper(key="date", freq="D")])
    .agg(daily_sales=("ext price", "sum"))
    .reset_index()
)
daily_sales.head()

In [ ]:
daily_sales["quarter_sales"] = daily_sales.groupby(
    pd.Grouper(key="date", freq="Q")
).agg({"daily_sales": "cumsum"})
daily_sales.head()

Чтобы получить хорошее представление о том, что происходит, вам нужно взглянуть на границу квартала (с конца марта по начало апреля):

<img src="https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/cumulative_total.png" >

Если вы хотите просто получить совокупный (cumulative) квартальный итог, вы можете связать несколько функций `groupby`.

Сначала сгруппируйте ежедневные результаты, затем сгруппируйте эти результаты по кварталам и используйте кумулятивную сумму:

In [ ]:
# веселый пример :)

sales.groupby([pd.Grouper(key="date", freq="D")]).agg(
    daily_sales=("ext price", "sum")
).groupby(pd.Grouper(freq="Q")).agg({"daily_sales": "cumsum"}).rename(
    columns={"daily_sales": "quarterly_sales"}
)

В этом примере я включил именованный подход агрегации (named aggregation approach), чтобы переименовать переменную и уточнить, что теперь это ежедневные продажи. Затем я снова группирую и использую совокупную (cumulative) сумму, чтобы получить текущую сумму за квартал. Наконец, я переименовал столбец в квартальные продажи (quarterly sales).

По отзывам, на первый взгляд, это сложно понять. Однако, если выполните по шагам, т.е. построите функцию и будете проверять результаты на каждом шаге, то начнете понимать ее.

Не расстраивайтесь!

## Сглаживание иерархических индексов столбцов

По умолчанию pandas в сводном `DataFrame` создает иерархический индекс у столбца:

In [ ]:
df.groupby(["embark_town", "class"]).agg({"fare": ["sum", "mean"]}).round()

В какой-то момент в процессе анализа вы, вероятно, захотите «сгладить» (flatten) столбцы, чтобы получилась одна строка с именами.

Я обнаружил, что мне лучше всего подходит следующий подход. 

Я использую параметр `as_index=False` при группировке, а затем создаю новое имя свернутого (collapsed) столбца.

Вот код:

In [ ]:
multi_df = df.groupby(["embark_town", "class"], as_index=False).agg(
    {"fare": ["sum", "mean"]}
)
multi_df

In [ ]:
multi_df.columns = ["_".join(col).rstrip("_") for col in multi_df.columns.values]
multi_df.round(2)

Вот изображение, показывающее, как выглядит сплющенный кадр данных:

<img src="https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/column_flatten.png" >

Я предпочитаю использовать `_` в качестве разделителя, но вы можете использовать другие значения. Просто имейте в виду, что для последующего анализа будет проще, если в именах результирующих столбцов нет пробелов.

## Промежуточные итоги

Если вы хотите добавить промежуточные итоги (subtotal), я рекомендую пакет [`sidetable`](https://github.com/chris1610/sidetable).

Инструкция по работе с `sidetable` на русском языке [тут](http://dfedorov.spb.ru/pandas/%D0%A1%D0%B2%D0%BE%D0%B4%D0%BD%D0%B0%D1%8F%20%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0%20%D0%B2%20pandas.html).

Вот как вы можете суммировать `fares` по `class`, `embark_town` и `sex` с промежуточным итогом на каждом уровне, а также общим итогом внизу:

In [ ]:
# pip install sidetable

In [ ]:
df.groupby(["class", "embark_town", "sex"]).agg({"fare": "sum"}).stb.subtotal()

`sidetable` также позволяет настраивать уровни промежуточных итогов и итоговые метки. Обратитесь к [документации пакета](https://github.com/chris1610/sidetable) для получения дополнительных примеров того, как `sidetable` может резюмировать данные.

## Резюме

Спасибо, что прочитали эту статью. Здесь много деталей, но это связано с тем, что существует множество различных применений для группировки и агрегирования данных с помощью pandas. Я надеюсь, что этот пост станет полезным ресурсом, который вы сможете добавить в закладки и вернуться к нему, когда столкнетесь с собственной сложной проблемой.

Если у вас есть другие распространенные техники, которые вы часто используете, дайте мне знать в комментариях к [статье](https://pbpython.com/groupby-agg.html). Если я получу что-нибудь полезное, я включу его в этот пост или как обновленную статью.